In [21]:
import numpy as np 
import pandas as pd 
import re
import math

## Data Load & ETL 

In [22]:
# player Stats 
player = pd.read_csv("./modified/nba_players_traditional_stats.csv")
win_share = pd.read_csv("./modified/nba_players_winshare_stats.csv")

# Salary
salary = pd.read_csv("./modified/nba_players_salary.csv")
salary_hoop = pd.read_csv("./modified/nba_players_salary_hoop.csv")

# Team Stat 
team_stats = pd.read_csv("./modified/nba_team_traditional_stats.csv")

In [23]:
salary['SALARY'] = salary['SALARY'].str.replace(',', '')
salary['SALARY'] = salary['SALARY'].str.replace('$', '')
salary['SALARY'] = salary['SALARY'].astype(int)
salary.rename(columns={"NAME":"PLAYER"}, inplace = True)

salary_hoop['SALARY'] = salary_hoop['SALARY'].str.replace(',', '')
salary_hoop['SALARY'] = salary_hoop['SALARY'].str.replace('$', '')
salary_hoop['SALARY'] = salary_hoop['SALARY'].astype(int)

salary_hoop['PLAYER'] = salary_hoop['PLAYER'].str.replace("'", '') 


In [24]:
player.head()

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,James Harden,HOU,28,72,59,13,35.4,30.4,9.0,20.1,...,5.4,8.8,4.4,1.8,0.7,2.3,53.0,31,4,7.3
1,Anthony Davis,NOP,25,75,45,30,36.4,28.1,10.4,19.5,...,11.1,2.3,2.2,1.5,2.6,2.1,55.1,50,1,3.9
2,LeBron James,CLE,33,82,50,32,36.9,27.5,10.5,19.3,...,8.6,9.1,4.2,1.4,0.9,1.7,54.1,52,18,1.3
3,Damian Lillard,POR,27,73,44,29,36.6,26.9,8.5,19.4,...,4.5,6.6,2.8,1.1,0.4,1.6,43.6,11,0,4.7
4,Giannis Antetokounmpo,MIL,23,75,39,36,36.7,26.9,9.9,18.7,...,10.0,4.8,3.0,1.5,1.4,3.1,51.7,42,1,2.0


In [25]:
salary_hoop.head()

,PLAYER,SALARY
0,Stephen Curry,34682550
1,LeBron James,33285709
2,Paul Millsap,30769231
3,Gordon Hayward,29727900
4,Blake Griffin,29512900


In [26]:
win_share.head()

,PLAYER,POS,AGE,TEAM,GAME,MP,PER,TS%,3PAR,FTR,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Alex Abrines,SG,24,OKC,75,1134,9.0,0.567,0.759,0.158,...,7.4,12.7,1.3,1.0,2.2,0.094,-0.5,-1.7,-2.2,-0.1
1,Quincy Acy,PF,27,BRK,70,1359,8.2,0.525,0.800,0.164,...,13.3,14.4,-0.1,1.1,1.0,0.036,-2.0,-0.2,-2.2,-0.1
2,Steven Adams,C,24,OKC,76,2487,20.6,0.630,0.003,0.402,...,13.2,16.7,6.7,3.0,9.7,0.187,2.2,1.1,3.3,3.3
3,Bam Adebayo,C,20,MIA,69,1368,15.7,0.570,0.021,0.526,...,13.6,15.9,2.3,1.9,4.2,0.148,-1.6,1.8,0.2,0.8
4,Arron Afflalo,SG,32,ORL,53,682,5.8,0.516,0.432,0.160,...,10.8,12.5,-0.1,0.2,0.1,0.009,-4.1,-1.8,-5.8,-0.7


In [27]:
## Remove Duplicated winshare becauce of Trade 
win_first = win_share.drop_duplicates(['PLAYER'],keep='first').copy()
win_last = win_share.drop_duplicates(['PLAYER'],keep='last').copy()
win_first.columns

p_win_share = pd.merge(win_first, win_last, on='PLAYER', suffixes=('', '_y'))
p_win_share['TEAM'] = p_win_share['TEAM_y']

p_win_share = p_win_share[win_first.columns]
p_win_share[p_win_share['PLAYER'] == 'Troy Williams']


,PLAYER,POS,AGE,TEAM,GAME,MP,PER,TS%,3PAR,FTR,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
522,Troy Williams,SF,23,NYK,21,307,15.1,0.544,0.349,0.275,...,13.5,20.1,0.2,0.4,0.5,0.085,-0.1,0.2,0.0,0.2


In [28]:
salary.rename(columns={"NAME":"PLAYER"}, inplace = True)
salary.head()

,PLAYER,TEAM,SALARY
0,Stephen Curry,Golden State Warriors,37457154
1,Blake Griffin,LA Clippers,32088932
2,Paul Millsap,Denver Nuggets,31269231
3,Kyle Lowry,Toronto Raptors,31200000
4,Gordon Hayward,Boston Celtics,29727900


In [29]:
player.head()

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,James Harden,HOU,28,72,59,13,35.4,30.4,9.0,20.1,...,5.4,8.8,4.4,1.8,0.7,2.3,53.0,31,4,7.3
1,Anthony Davis,NOP,25,75,45,30,36.4,28.1,10.4,19.5,...,11.1,2.3,2.2,1.5,2.6,2.1,55.1,50,1,3.9
2,LeBron James,CLE,33,82,50,32,36.9,27.5,10.5,19.3,...,8.6,9.1,4.2,1.4,0.9,1.7,54.1,52,18,1.3
3,Damian Lillard,POR,27,73,44,29,36.6,26.9,8.5,19.4,...,4.5,6.6,2.8,1.1,0.4,1.6,43.6,11,0,4.7
4,Giannis Antetokounmpo,MIL,23,75,39,36,36.7,26.9,9.9,18.7,...,10.0,4.8,3.0,1.5,1.4,3.1,51.7,42,1,2.0


## Step1. Player Salary Join ( Stats & Salary )

In [30]:
# Merge Data 
player_salary  = pd.merge(player, salary, how = 'left', on='PLAYER') # 1st fill in the salary data
player_salary = pd.merge(player_salary, salary_hoop,how = 'left' , on='PLAYER' ) # 2nd fill in the salary data 

In [31]:
player_salary = pd.merge(player_salary, salary_hoop,how = 'left' , on='PLAYER' )

In [32]:
## Salary NAN to 0 
player_salary['SALARY'] = list(map(lambda x : 0 if math.isnan(x) else x , player_salary['SALARY']))
player_salary = player_salary.drop('SALARY_x', axis=1)
player_salary = player_salary.drop('SALARY_y', axis=1)

In [33]:
player_salary[player_salary['SALARY'] ==0    ][['PLAYER','TEAM_x','SALARY']]

,PLAYER,TEAM_x,SALARY
44,Tim Hardaway Jr.,NYK,0.0
63,D'Angelo Russell,BKN,0.0
118,E'Twaun Moore,NOP,0.0
140,De'Aaron Fox,SAC,0.0
252,Kyle O'Quinn,NYK,0.0
279,Nene,HOU,0.0
309,Xavier Rathan-Mayes,MEM,0.0
312,Timothe Luwawu-Cabarrot,PHI,0.0
321,C.J. Williams,LAC,0.0
325,Wade Baldwin IV,POR,0.0


## Step2 Player Win Share Join ( Player & Win share ) 

In [34]:
player_salary.head(5)

,PLAYER,TEAM_x,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,TEAM_y,SALARY
0,James Harden,HOU,28,72,59,13,35.4,30.4,9.0,20.1,...,4.4,1.8,0.7,2.3,53.0,31,4,7.3,Houston Rockets,28299399.0
1,Anthony Davis,NOP,25,75,45,30,36.4,28.1,10.4,19.5,...,2.2,1.5,2.6,2.1,55.1,50,1,3.9,New Orleans Pelicans,23775506.0
2,LeBron James,CLE,33,82,50,32,36.9,27.5,10.5,19.3,...,4.2,1.4,0.9,1.7,54.1,52,18,1.3,NaN,33285709.0
3,Damian Lillard,POR,27,73,44,29,36.6,26.9,8.5,19.4,...,2.8,1.1,0.4,1.6,43.6,11,0,4.7,Portland Trail Blazers,26153057.0
4,Giannis Antetokounmpo,MIL,23,75,39,36,36.7,26.9,9.9,18.7,...,3.0,1.5,1.4,3.1,51.7,42,1,2.0,Milwaukee Bucks,22471911.0


In [35]:
season_record = pd.merge(player_salary,p_win_share,how = 'left' , on='PLAYER' )

In [36]:

season_record.to_csv("./modified/season_record.csv", mode='w',index=False)


In [37]:
season_record[0:10][['PLAYER','SALARY','WS']]

,PLAYER,SALARY,WS
0,James Harden,28299399.0,15.4
1,Anthony Davis,23775506.0,13.7
2,LeBron James,33285709.0,14.0
3,Damian Lillard,26153057.0,12.6
4,Giannis Antetokounmpo,22471911.0,11.9
5,Stephen Curry,34682550.0,9.1
6,Kevin Durant,25000000.0,10.4
7,Russell Westbrook,28299399.0,10.1
8,DeMarcus Cousins,18063850.0,4.7
9,Devin Booker,2319360.0,2.4


In [38]:
season_record['WPS'] =   season_record['WS'] / season_record['SALARY'] 
season_record.to_csv("./modified/season_record_2.csv", mode='w',index=False)

In [39]:
pd.options.display.float_format = '{:.10f}'.format
#season_record[["PLAYER", "EFF"]].sort_values(by=['EFF'], ascending=False)
season_record[["PLAYER", "EFF"]].sort_values(by=['EFF'], ascending=False)
#df.sort_values(by=['col1'])

KeyError: "['EFF'] not in index"

## Visualization 

In [40]:
season_record

,PLAYER,TEAM_x,AGE_x,GP,W,L,MIN,PTS,FGM,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,WPS
0,James Harden,HOU,28,72,59,13,35.4000000000,30.4000000000,9.0000000000,20.1000000000,...,36.1000000000,11.6000000000,3.8000000000,15.4000000000,0.2890000000,9.6000000000,1.3000000000,10.9000000000,8.3000000000,0.0000005442
1,Anthony Davis,NOP,25,75,45,30,36.4000000000,28.1000000000,10.4000000000,19.5000000000,...,30.0000000000,8.8000000000,4.9000000000,13.7000000000,0.2410000000,2.8000000000,2.3000000000,5.2000000000,4.9000000000,0.0000005762
2,LeBron James,CLE,33,82,50,32,36.9000000000,27.5000000000,10.5000000000,19.3000000000,...,31.6000000000,11.0000000000,3.0000000000,14.0000000000,0.2210000000,7.6000000000,2.0000000000,9.6000000000,8.9000000000,0.0000004206
3,Damian Lillard,POR,27,73,44,29,36.6000000000,26.9000000000,8.5000000000,19.4000000000,...,30.6000000000,9.9000000000,2.7000000000,12.6000000000,0.2270000000,7.7000000000,-1.0000000000,6.7000000000,5.9000000000,0.0000004818
4,Giannis Antetokounmpo,MIL,23,75,39,36,36.7000000000,26.9000000000,9.9000000000,18.7000000000,...,31.2000000000,8.3000000000,3.6000000000,11.9000000000,0.2070000000,3.9000000000,1.9000000000,5.8000000000,5.4000000000,0.0000005295
5,Stephen Curry,GSW,30,51,41,10,32.0000000000,26.4000000000,8.4000000000,16.9000000000,...,31.0000000000,7.2000000000,1.8000000000,9.1000000000,0.2670000000,9.9000000000,-1.3000000000,8.6000000000,4.4000000000,0.0000002624
6,Kevin Durant,GSW,29,68,49,19,34.2000000000,26.4000000000,9.3000000000,18.0000000000,...,30.4000000000,7.5000000000,2.9000000000,10.4000000000,0.2150000000,5.0000000000,0.7000000000,5.6000000000,4.5000000000,0.0000004160
7,Russell Westbrook,OKC,29,80,47,33,36.4000000000,25.4000000000,9.5000000000,21.1000000000,...,34.1000000000,5.5000000000,4.5000000000,10.1000000000,0.1660000000,5.1000000000,3.1000000000,8.2000000000,7.5000000000,0.0000003569
8,DeMarcus Cousins,NOP,27,48,27,21,36.2000000000,25.2000000000,8.5000000000,18.0000000000,...,31.9000000000,1.5000000000,3.2000000000,4.7000000000,0.1300000000,1.9000000000,3.6000000000,5.5000000000,3.3000000000,0.0000002602
9,Devin Booker,PHX,21,54,16,38,34.5000000000,24.9000000000,8.4000000000,19.5000000000,...,31.7000000000,1.9000000000,0.6000000000,2.4000000000,0.0630000000,2.9000000000,-2.4000000000,0.4000000000,1.2000000000,0.0000010348
